In [1]:
import os
import pandas as pd
import torch
import gc
import numpy as np
import os
from dotenv import load_dotenv
from datetime import datetime
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    EarlyStoppingCallback
)
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from peft import LoraConfig, get_peft_model, TaskType

# Load environment variables from .env file
load_dotenv()

# Get token from environment
HF_TOKEN = os.environ.get("HUGGINGFACE_TOKEN")
if not HF_TOKEN:
    raise ValueError("Please set the HUGGINGFACE_TOKEN environment variable")
os.environ["HF_TOKEN"] = HF_TOKEN

class MorphologyZeroShotDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.examples = []
        
        # Process each row in the dataset
        for _, row in data.iterrows():
            # Create zero-shot prompt (direct question without examples)
            input_text = self.create_zero_shot_prompt(row)
            
            # Create expected output
            target_text = self.create_target_output(row)
            
            # Combine for full text
            full_text = f"{input_text}{target_text}</s>"
            
            # Store the example
            self.examples.append(full_text)
            
        print(f"Created {len(self.examples)} zero-shot training examples")
    
    def create_zero_shot_prompt(self, row):
        """Create a prompt without any examples (zero-shot)"""
        # Main instruction
        instruction = (
            "You are answering a morphology question. "
            "Begin your answer with 'Answer: Choice X' for multiple choice or 'Answer: [text]' for open-ended questions. "
            "Provide a clear explanation afterward."
        )
        
        # Determine if it's multiple choice
        task_type = "Multiple choice" if pd.notna(row.get('Choice_1', pd.NA)) else "Open-ended"
        
        # Format the input
        input_text = (
            f"{instruction}\n\n"
            f"# Question Information\n"
            f"- Type: {task_type}\n"
            f"- Task: {row['Task']}\n"
            f"- Category: {row['Category']}\n"
            f"- Word: {row['Word']}\n"
            f"- Question: {row['Instruction']}\n"
        )
        
        # Add choices if multiple choice
        if pd.notna(row.get('Choice_1', pd.NA)):
            input_text += "\n# Available Choices\n"
            choice_num = 1
            while True:
                choice_key = f'Choice_{choice_num}'
                if choice_key not in row or pd.isna(row[choice_key]):
                    break
                input_text += f"- Choice {choice_num}: {row[choice_key]}\n"
                choice_num += 1
        
        # Add separator
        input_text += "\n# Your Answer:\n"
        
        return input_text
    
    def create_target_output(self, row):
        """Create the expected model output"""
        if pd.notna(row.get('Choice_1', pd.NA)):
            correct_choice = row[f'Choice_{row["Correct_Answer"]}']
            target_text = (
                f"Answer: Choice {row['Correct_Answer']}. "
                f"{correct_choice} is correct because it demonstrates the {row['Category'].lower()} "
                f"concept. In the word '{row['Word']}', we can identify the {row['Task'].lower()} "
                f"through proper morphological analysis. This is a key concept in understanding "
                f"how words are formed and structured in English."
            )
        else:
            target_text = (
                f"Answer: {str(row['Correct_Answer'])}. "
                f"This demonstrates the {row['Category'].lower()} concept in '{row['Word']}'. "
                f"When analyzing how this word {row['Task'].lower()}, we can see the morphological "
                f"principles at work. This helps us understand the structure and formation of words."
            )
        
        return target_text
    
    def __len__(self):
        return len(self.examples)
    
    def __getitem__(self, idx):
        text = self.examples[idx]
        
        # Tokenize the text
        encodings = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        # Separate the input part and create labels
        input_part = self.tokenizer(
            text.split("# Your Answer:")[0] + "# Your Answer:\n",
            add_special_tokens=False,
            return_tensors="pt"
        )
        
        # Get length of the input part
        input_length = input_part['input_ids'].shape[1]
        
        # Create labels, masking the input part
        labels = encodings['input_ids'].clone()
        labels[:, :input_length] = -100  # Ignore loss for the input part
        
        return {
            'input_ids': encodings['input_ids'].squeeze(),
            'attention_mask': encodings['attention_mask'].squeeze(),
            'labels': labels.squeeze()
        }

def prepare_data(csv_path):
    """Load and prepare the data for training"""
    df = pd.read_csv(csv_path)
    
    # Convert necessary columns to string
    for col in ['Correct_Answer', 'Word_Difficulty', 'Task_Difficulty']:
        if col in df.columns:
            df[col] = df[col].astype(str)
    
    # Split data with stratification if possible
    try:
        train_df, val_df = train_test_split(
            df, 
            test_size=0.15,
            random_state=42,
            stratify=df[['Category', 'Task']].apply(lambda x: f"{x['Category']}_{x['Task']}", axis=1)
        )
    except ValueError:
        # Fallback to stratifying by just Category
        train_df, val_df = train_test_split(
            df, 
            test_size=0.15, 
            random_state=42, 
            stratify=df['Category']
        )
    
    print(f"Original data size: {len(df)}")
    print(f"Training set size: {len(train_df)}, Validation set size: {len(val_df)}")
    
    return train_df, val_df

def monitor_gpu_memory(message):
    """Helper function to monitor GPU memory usage"""
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / (1024 ** 3)
        max_allocated = torch.cuda.max_memory_allocated() / (1024 ** 3)
        reserved = torch.cuda.memory_reserved() / (1024 ** 3)
        print(f"{message}: Allocated: {allocated:.2f} GB, Max: {max_allocated:.2f} GB, Reserved: {reserved:.2f} GB")

def save_config_locally(model_name, output_dir):
    """Save model config locally to avoid authentication issues during training"""
    from huggingface_hub import hf_hub_download
    import shutil
    
    # Create directory structure
    base_path = os.path.join(output_dir, "base_model_config")
    os.makedirs(base_path, exist_ok=True)
    
    try:
        # Download config file
        config_path = hf_hub_download(
            repo_id=model_name,
            filename="config.json",
            token=HF_TOKEN
        )
        
        # Copy to our directory
        shutil.copy(config_path, os.path.join(base_path, "config.json"))
        print(f"Config saved locally to {base_path}")
        
        return True
    except Exception as e:
        print(f"Error saving config locally: {e}")
        return False

def train_zero_shot_gemma(train_df, val_df, model_name="google/gemma-2b-it", output_dir="gemma_zero_shot", use_lora=True):
    """Fine-tune Gemma model with zero-shot approach"""
    # Clean memory
    gc.collect()
    torch.cuda.empty_cache()
    
    monitor_gpu_memory("Initial GPU state")
    
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    
    # Save config locally to avoid authentication issues
    save_config_locally(model_name, output_dir)
    
    # Load tokenizer
    print("Loading tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(model_name, token=HF_TOKEN)
    
    # Save tokenizer locally
    tokenizer_save_path = os.path.join(output_dir, "tokenizer")
    tokenizer.save_pretrained(tokenizer_save_path)
    
    gc.collect()
    torch.cuda.empty_cache()
    
    # Load model
    print("Loading model...")
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        token=HF_TOKEN,
        low_cpu_mem_usage=True,
        torch_dtype=torch.float32,  # Use float32 to avoid gradient issues
        device_map="auto"
    )
    
    monitor_gpu_memory("After model loading")
    
    # Apply LoRA for efficient fine-tuning if requested
    if use_lora:
        print("Applying LoRA adapters...")
        # Configure LoRA
        lora_config = LoraConfig(
            r=16,  # rank
            lora_alpha=32,  # scaling factor
            target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
            lora_dropout=0.05,
            bias="none",
            task_type=TaskType.CAUSAL_LM
        )
        
        # Apply LoRA to model
        model = get_peft_model(model, lora_config)
        model.print_trainable_parameters()
    else:
        # Enable gradient checkpointing for full fine-tuning
        model.gradient_checkpointing_enable(gradient_checkpointing_kwargs={"use_reentrant": False})
    
    monitor_gpu_memory("After model adaptation")
    
    # Create zero-shot datasets
    print("Creating zero-shot datasets...")
    train_dataset = MorphologyZeroShotDataset(train_df, tokenizer, max_length=512)
    val_dataset = MorphologyZeroShotDataset(val_df, tokenizer, max_length=512)
    
    monitor_gpu_memory("After dataset creation")
    
    # Training arguments
    batch_size = 2 if use_lora else 1
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=12,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        warmup_ratio=0.1,
        learning_rate=3e-5 if use_lora else 2e-5,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10,
        evaluation_strategy="steps",
        eval_steps=30,
        save_steps=30,
        save_total_limit=3,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        gradient_accumulation_steps=4 if use_lora else 8,
        fp16=False,  # Disable fp16 to avoid gradient issues
        bf16=False,
        max_grad_norm=1.0,
        lr_scheduler_type="cosine",
        optim="adamw_torch",
        gradient_checkpointing=not use_lora,  # Enable for full fine-tuning only
        ddp_find_unused_parameters=False,
        dataloader_pin_memory=False,
        report_to="none",  # Disable reporting to save memory
        run_name=f"gemma_zero_shot_{datetime.now().strftime('%Y%m%d_%H%M')}",
        hub_token=HF_TOKEN,
    )
    
    # Data collator
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False
    )
    
    monitor_gpu_memory("Before trainer initialization")
    
    # Initialize trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        data_collator=data_collator,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )
    
    monitor_gpu_memory("After trainer initialization")
    
    # Train the model
    print("Starting training...")
    trainer.train()
    
    # Save the model
    print("Saving model...")
    model_save_path = os.path.join(output_dir, "final_model")
    
    if use_lora:
        # For LoRA, save the adapter
        model.save_pretrained(model_save_path, token=HF_TOKEN)
    else:
        # For full model, save everything
        trainer.save_model(model_save_path)
    
    # Save tokenizer from local copy
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_save_path)
    tokenizer.save_pretrained(model_save_path)
    
    print(f"Model saved to {model_save_path}")
    
    return model, tokenizer

def test_model(model, tokenizer, test_questions):
    """Test the model on sample questions"""
    model.eval()
    results = []
    
    for question in test_questions:
        # Create zero-shot prompt
        instruction = (
            "You are answering a morphology question. "
            "Begin your answer with 'Answer: Choice X' for multiple choice or 'Answer: [text]' for open-ended questions. "
            "Provide a clear explanation afterward."
        )
        
        # Format input
        input_text = (
            f"{instruction}\n\n"
            f"# Question Information\n"
            f"- Type: Multiple choice\n"
            f"- Task: {question['Task']}\n"
            f"- Category: {question['Category']}\n"
            f"- Word: {question['Word']}\n"
            f"- Question: {question['Instruction']}\n"
            f"\n# Available Choices\n"
        )
        
        # Add choices
        for i, choice in enumerate(question['Choices'], 1):
            input_text += f"- Choice {i}: {choice}\n"
        
        # Add answer prompt
        input_text += "\n# Your Answer:\n"
        
        # Generation settings
        generation_config = {
            'max_new_tokens': 200,
            'do_sample': True,
            'temperature': 0.7,
            'top_p': 0.92,
            'top_k': 50,
            'repetition_penalty': 1.2,
        }
        
        # Generate answer
        inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
        
        with torch.no_grad():
            outputs = model.generate(
                input_ids=inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                **generation_config
            )
        
        # Decode output
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Extract answer part
        if input_text in generated_text:
            answer = generated_text[len(input_text):].strip()
        else:
            answer = generated_text
            
        results.append({
            'question': question,
            'input': input_text,
            'generated': answer
        })
    
    return results

def main():
    # Configure paths
    data_path = 'Data/MC_data_MMA.csv'
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    output_dir = f"gemma_zero_shot_{timestamp}"
    
    # Display GPU information
    print(f"CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"GPU device: {torch.cuda.get_device_name(0)}")
        print(f"Total GPU memory: {torch.cuda.get_device_properties(0).total_memory / (1024**3):.2f} GB")
    
    # Prepare data
    print("\nPreparing data...")
    train_df, val_df = prepare_data(data_path)
    
    # Define whether to use LoRA (recommended for better memory efficiency)
    use_lora = True  # Set to False for full fine-tuning
    
    # Train model with zero-shot approach
    print(f"\nTraining Gemma model with zero-shot approach...")
    model, tokenizer = train_zero_shot_gemma(train_df, val_df, output_dir=output_dir, use_lora=use_lora)
    
    # Test questions
    test_questions = [
        {
            "Task": "Identify",
            "Category": "Derivation",
            "Word": "happiness",
            "Instruction": "What is the base word and suffix in 'happiness'?",
            "Choices": [
                "base: happy, suffix: -ness",
                "base: happ, suffix: -iness",
                "base: happi, suffix: -ness",
                "base: hap, suffix: -piness"
            ]
        },
        {
            "Task": "Analyze",
            "Category": "Compounding",
            "Word": "blackboard",
            "Instruction": "Identify the type of compound word in 'blackboard'.",
            "Choices": [
                "Endocentric compound",
                "Exocentric compound",
                "Copulative compound",
                "Appositional compound"
            ]
        }
    ]
    
    # Test the model
    print("\nTesting model on sample questions...")
    results = test_model(model, tokenizer, test_questions)
    
    # Print results
    print("\nSample Generated Answers:")
    for i, result in enumerate(results, 1):
        print(f"\nExample {i}:")
        print(f"Question: {result['question']['Instruction']}")
        print(f"Word: {result['question']['Word']}")
        print(f"Generated Answer: {result['generated']}")
        print("-" * 80)

if __name__ == "__main__":
    # Clean memory before starting
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()
    main()

CUDA available: True
GPU device: NVIDIA A100-SXM4-80GB
Total GPU memory: 79.14 GB

Preparing data...
Original data size: 268
Training set size: 227, Validation set size: 41

Training Gemma model with zero-shot approach...
Initial GPU state: Allocated: 0.00 GB, Max: 0.00 GB, Reserved: 0.00 GB
Config saved locally to gemma_zero_shot_20250327_162909/base_model_config
Loading tokenizer...
Loading model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

After model loading: Allocated: 5.64 GB, Max: 5.64 GB, Reserved: 5.65 GB
Applying LoRA adapters...


Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


trainable params: 3,686,400 || all params: 2,509,858,816 || trainable%: 0.14687678751090355
After model adaptation: Allocated: 5.65 GB, Max: 5.65 GB, Reserved: 5.65 GB
Creating zero-shot datasets...
Created 227 zero-shot training examples
Created 41 zero-shot training examples
After dataset creation: Allocated: 5.65 GB, Max: 5.65 GB, Reserved: 5.65 GB
Before trainer initialization: Allocated: 5.65 GB, Max: 5.65 GB, Reserved: 5.65 GB
After trainer initialization: Allocated: 5.65 GB, Max: 5.65 GB, Reserved: 5.65 GB
Starting training...


Step,Training Loss,Validation Loss
30,3.514400,3.152705
60,1.667100,1.447751
90,0.636500,0.570354
120,0.346100,0.350446
150,0.279000,0.312200
180,0.264100,0.295596
210,0.259200,0.284656
240,0.260400,0.280846
270,0.248200,0.277584
300,0.239400,0.276542


Saving model...
Model saved to gemma_zero_shot_20250327_162909/final_model

Testing model on sample questions...

Sample Generated Answers:

Example 1:
Question: What is the base word and suffix in 'happiness'?
Word: happiness
Generated Answer: Answer: Choice 1. base: happy, suffix: -ness is correct because it demonstrates the concept of derivation through proper identification of the key words in the vocabulary/word form. This is crucial because it</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s